<a href="https://colab.research.google.com/github/kirat89/Google_colab/blob/main/Audiobook_train_test(ann).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Libraries


In [1]:
import numpy as np

## Loading the Data

In [2]:
raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter=',')
raw_inputs=raw_csv_data[:,1:-1]
targets=raw_csv_data[:,-1]

In [3]:
print(raw_inputs)
print(targets)

[[1620.   1620.     19.73 ... 1603.8     5.     92.  ]
 [2160.   2160.      5.33 ...    0.      0.      0.  ]
 [2160.   2160.      5.33 ...    0.      0.    388.  ]
 ...
 [2160.   2160.      6.14 ...    0.      0.      0.  ]
 [1620.   1620.      5.33 ...  615.6     0.     90.  ]
 [1674.   3348.      5.33 ...    0.      0.      0.  ]]
[0. 0. 0. ... 0. 0. 1.]


## Shuffle the Dataset


In [4]:
indices=np.arange(raw_inputs.shape[0])
print(indices)



[    0     1     2 ... 14081 14082 14083]


In [5]:
np.random.shuffle(indices)

In [6]:
shuffled_inputs=raw_inputs[indices]
shuffled_targets=targets[indices]

In [7]:
print(shuffled_inputs)

[[1.620e+03 1.620e+03 5.610e+00 ... 0.000e+00 0.000e+00 1.000e+00]
 [3.240e+02 3.240e+02 9.220e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.620e+03 1.620e+03 1.973e+01 ... 5.670e+02 0.000e+00 2.100e+01]
 ...
 [2.160e+03 2.160e+03 8.000e+00 ... 4.104e+02 0.000e+00 2.800e+01]
 [1.620e+03 1.620e+03 5.330e+00 ... 0.000e+00 0.000e+00 3.300e+01]
 [2.160e+03 2.160e+03 5.690e+00 ... 0.000e+00 0.000e+00 0.000e+00]]


## Balance the Dataset


In [8]:
#counting the no's of one in data
num_one_target=int(np.sum(shuffled_targets))
#setting a counter for 0 in targets
zero_counter=0
#to remove extra data we create a list that will contain the indices
indices_to_remove=[]
# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(shuffled_targets.shape[0]):
  if shuffled_targets[i]==0:
    zero_counter+=1
    if zero_counter>num_one_target:
      indices_to_remove.append(i)


balanced_shuffled_inputs=np.delete(shuffled_inputs,indices_to_remove,axis=0)
balanced_shuffled_targets=np.delete(shuffled_targets,indices_to_remove,axis=0)

## Train Test Validation Split

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(balanced_shuffled_inputs,balanced_shuffled_targets,test_size=.2,random_state=1)

## Scaling The Dataset

In [10]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.transform(x_test)

## Deep Learning Model

In [11]:
import tensorflow as tf
ann=tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=100,activation='relu'))
ann.add(tf.keras.layers.Dense(units=100,activation='relu'))

ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [12]:
ann.compile(optimizer='adam' ,loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
ann.fit(x_train,y_train,batch_size=32,epochs=100,verbose=2)

Epoch 1/100
112/112 - 1s - loss: 0.4799 - accuracy: 0.7628
Epoch 2/100
112/112 - 0s - loss: 0.3984 - accuracy: 0.7969
Epoch 3/100
112/112 - 0s - loss: 0.3811 - accuracy: 0.8066
Epoch 4/100
112/112 - 0s - loss: 0.3712 - accuracy: 0.8011
Epoch 5/100
112/112 - 0s - loss: 0.3709 - accuracy: 0.8050
Epoch 6/100
112/112 - 0s - loss: 0.3609 - accuracy: 0.8128
Epoch 7/100
112/112 - 0s - loss: 0.3607 - accuracy: 0.8159
Epoch 8/100
112/112 - 0s - loss: 0.3557 - accuracy: 0.8136
Epoch 9/100
112/112 - 0s - loss: 0.3567 - accuracy: 0.8111
Epoch 10/100
112/112 - 0s - loss: 0.3520 - accuracy: 0.8187
Epoch 11/100
112/112 - 0s - loss: 0.3538 - accuracy: 0.8108
Epoch 12/100
112/112 - 0s - loss: 0.3537 - accuracy: 0.8142
Epoch 13/100
112/112 - 0s - loss: 0.3479 - accuracy: 0.8198
Epoch 14/100
112/112 - 0s - loss: 0.3483 - accuracy: 0.8175
Epoch 15/100
112/112 - 0s - loss: 0.3575 - accuracy: 0.8089
Epoch 16/100
112/112 - 0s - loss: 0.3507 - accuracy: 0.8170
Epoch 17/100
112/112 - 0s - loss: 0.3516 - accura

## Calculating Accuracy

In [14]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

pred=ann.predict(x_test)
pred=(pred>.5)
cm=confusion_matrix(y_test,pred)
print(cm)

[[403  40]
 [113 339]]


In [15]:
score=accuracy_score(y_test,pred)
print(score)

0.829050279329609


In [16]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         0.0       0.78      0.91      0.84       443
         1.0       0.89      0.75      0.82       452

    accuracy                           0.83       895
   macro avg       0.84      0.83      0.83       895
weighted avg       0.84      0.83      0.83       895

